In [ ]:
#client.drop_database("stockDB")

In [13]:
#function that grabs sector change data from google finance
import requests
from bs4 import BeautifulSoup

# function used to find biggest_gainer within a certain sector
def gainer(url):
    gainer={'change': '', 'equity': ''}
    response=requests.get(url)
    sec_page=BeautifulSoup(response.content, 'lxml')
    table=sec_page.find('table', class_='topmovers')
    
    for tr in table.find_all('tr'):
        if tr.find('a'):
            gainer['equity'] = tr.find('a').get_text()
        if tr.find('span'):
            tag=tr.find('span')
            thing=tag.get_text()
            gainer['change'] = float(thing)
            break
            
    if gainer['change']<0:
        gainer={}
    return gainer

# function used to find biggest_loser within a certain sector
def loser(url):
    loser = {'change': '', 'equity': ''}
    response = requests.get(url)
    sec_page = BeautifulSoup(response.content, 'lxml')
    whole_table = sec_page.find('table', class_='topmovers')
    
    table = whole_table.find('tr', class_='beginsec').next_sibling
    loser['equity'] = table.find('a').get_text()
    thing = table.find('span').get_text()
    loser['change'] = float(thing)
    return(loser)

# function used to report the whole market
def google_sector_report():
    import requests
    from bs4 import BeautifulSoup
    import json

    url='https://www.google.com/finance'
    status='BAD'
    report={'status': status}
    result=dict()
    response=requests.get(url)
    main_page=BeautifulSoup(response.content, 'lxml')
    div=main_page.find('div', {'id': 'secperf'})

    for tr in div.find_all('tr'):
        if tr.find('a'):
            sec_name=tr.find('a').get_text()
            if sec_name=='Cyclical Cons. Goods ...':
                sec_name = 'Cyclical Cons Goods'
            if sec_name=='Non-Cyclical Cons. Goods...':
                sec_name = 'Non-Cyclical Cons Goods'
            if sec_name=='Telecommunications Servi...':
                sec_name = 'Telecommunications Servi'
            sec_link='https://www.google.com' + tr.find('a').get('href')
            sec_gainer=gainer(sec_link)
            sec_loser=loser(sec_link)

            new_sec={sec_name: {'change': '', 'biggest_gainer':sec_gainer, 'biggest_loser':sec_loser}}
            result.update(new_sec)

        if tr.find('span'):
            sec_change=tr.find('span').get_text()
            sec_change=float(sec_change.strip('%'))

            result[sec_name]['change']=sec_change
            
    status='GOOD'
    report={'status': status, 'result': result}
    json_report = json.dumps(report)
    return(json_report)

In [14]:
#saving the JSON object (only the result part) in a file 
import json
from datetime import datetime

d = datetime.today()
d = d.strftime('%Y-%m-%d')

with open(d+'.txt', 'w') as f:
    today = json.loads(google_sector_report())
    today = today['result']
    date_field = {"date":d}   #add an additional date field
    today.update(date_field)
    today = json.dumps(today)
    f.write(today)

run mongod.exe

In [15]:
# Creat MongoDB collection named stock
from pymongo import MongoClient
client = MongoClient('127.0.0.1', 27017)
db = client.stockDB
stock = db.s

In [16]:
#stores the data 
import json
with open('2016-11-10.txt', 'r') as f:
    data = json.loads(f.read())
stock.insert_one(data)

with open('2016-11-11.txt', 'r') as f:
    data = json.loads(f.read())
stock.insert_one(data)

with open('2016-11-12.txt', 'r') as f:
    data = json.loads(f.read())
stock.insert_one(data)

In [17]:
#function that updates the collection as new data arrives
import json
today = json.loads(google_sector_report())
today = today['result']

from datetime import datetime
d = datetime.today()
d = d.strftime('%Y-%m-%d')
date_field = {"date":d}
today.update(date_field)

stock.insert_one(today)

In [19]:
#show the collection 
from pprint import pprint

for daily_data in stock.find():
    pprint(daily_data['date'])

'2016-11-10'
'2016-11-11'
'2016-11-12'
'2016-11-14'


In [20]:
#interface object: stock_analysis
class stock_analysis:

    def __init__(self):
        def get_sec_list(sector):
            l = []
            for daily_data in stock.find():
                change = daily_data[sector]['change']
                date = daily_data['date']
                l.append((date,change))
            return l 
        #self.sec_list = get_sec_list(sector)
    
    def daily_sec_max(self, date):
        from operator import itemgetter
        daily_data = stock.find_one({"date": date})

        d=dict()
        for sector in daily_data:
            if sector != '_id' and sector != 'date': 
                sector_move = abs(daily_data[sector]['change'])
                d[sector] = sector_move
        return(sorted(d.items(),key = itemgetter(1), reverse = True)[0])
    
    def sec_max_ever(self):
        from operator import itemgetter
        d=dict()
        for daily_data in stock.find():
            for sector in daily_data:
                if sector != '_id' and sector != 'date':
                    d[sector] = 0

            if d:
                break

        for daily_data in stock.find():
            for sector in daily_data:
                if sector != '_id' and sector != 'date':
                    daily_sec_move = abs(daily_data[sector]['change'])
                    if daily_sec_move > d[sector]:
                        d[sector]=daily_sec_move
        return(sorted(d.items(), key = itemgetter(1), reverse = True)[0])
    
    def daily_stock_max(self, date):
        from operator import itemgetter
        test_data = stock.find_one({"date": date})
        move_list={}
        for key in test_data:
            if key != '_id' and key != 'date':
                try:
                    a=test_data[key]['biggest_gainer']['change']
                    try:
                        b=abs(test_data[key]['biggest_loser']['change'])
                        move_list[test_data[key]['biggest_loser']['equity']]=max(a,b)
                    except:
                        move_list[test_data[key]['biggest_loser']['equity']]=a
                except:
                    try:
                        b=abs(test_data[key]['biggest_loser']['change'])
                        move_list[test_data[key]['biggest_loser']['equity']]=b
                    except:
                        continue
        sorted_list=sorted(move_list.items(), key=itemgetter(1),reverse=True)
        return sorted_list[0]
    
    def stock_max_ever(self):
        from operator import itemgetter
        move_list={}
        for date in stock.find():
            test_data = date
            for key in test_data:
                if key != '_id' and key != 'date':
                    try:
                        a=test_data[key]['biggest_gainer']['change']
                        try:
                            b=abs(test_data[key]['biggest_loser']['change'])
                            move_list[test_data[key]['biggest_loser']['equity']]=max(a,b)
                        except:
                            move_list[test_data[key]['biggest_loser']['equity']]=a
                    except:
                        try:
                            b=abs(test_data[key]['biggest_loser']['change'])
                            move_list[test_data[key]['biggest_loser']['equity']]=b
                        except:
                            continue

        sorted_list=sorted(move_list.items(), key=itemgetter(1),reverse=True)
        return sorted_list[0]
    
    def get_sec_list(self, sector):
        l = []
        for daily_data in stock.find():
            change = daily_data[sector]['change']
            date = daily_data['date']
            l.append((date,change))
        return l 
    #average change for a given sector
    def sec_move_avg(self, sector):
        l1 = self.get_sec_list(sector)
        l2 = []
        for x in l1:
            l2.append(x[1])
        return sum(l2)/len(l2)
    #standard deviation of change for a given sector 
    def sec_move_stdev(self, sector):
        import statistics
        l1 = self.get_sec_list(sector)
        l2 = []
        for x in l1:
            l2.append(x[1])
        return(statistics.stdev(l2))
    #cumulative(compounded) movement for a given sector
    def sec_move_cum(self, sector):
        l1 = self.get_sec_list(sector)
        cum_change = 1
        for x in l1:
            cum_change = cum_change*(1+x[1]/100)
        cum_change = (cum_change-1)*100
        return(cum_change)
    #greatest daily movement for a given sector
    def sec_move_max(self, sector):
        from operator import itemgetter
        l1 = self.get_sec_list(sector)
        l2 = sorted(l1, key=itemgetter(1),reverse=True)
        return(l2[0])

In [21]:
#class instantiation
s=stock_analysis()
#A
s.daily_sec_max('2016-11-11')

('Energy', 1.99)

In [22]:
#B
s.sec_max_ever()

('Non-Cyclical Cons Goods', 2.39)

In [23]:
#C
s.daily_stock_max('2016-11-11')

('Photronics, Inc.', 21.79)

In [24]:
#D
s.stock_max_ever()

('Empire Resorts Inc', 22.07)

In [25]:
#E
s.get_sec_list('Energy') 

[('2016-11-10', -0.34),
 ('2016-11-11', -1.99),
 ('2016-11-12', -1.81),
 ('2016-11-14', 0.04)]

In [26]:
#F 4 other questions to this list
#average change for a given sector
s.sec_move_avg('Industrials')

0.5575000000000001

In [27]:
#standard deviation of change for a given sector
s.sec_move_stdev('Utilities')

0.9442280797914593

In [28]:
#cumulative movement for a given sector
s.sec_move_cum('Industrials')

2.244209546979481

In [29]:
#greatest daily movement for a given sector
s.sec_move_max('Industrials')

('2016-11-10', 1.35)